In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import time
import copy

# Configuration
DATA_DIR = './dataset_final'
MODEL_SAVE_PATH = 'triage_router.pth'
BATCH_SIZE = 32
NUM_EPOCHS = 20  # Increased to 20 to aim for >99% accuracy
INPUT_SIZE = 224

# Check device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [2]:
# Data Transforms
# We use standard ImageNet normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10), # Added rotation for better generalization
        transforms.ColorJitter(brightness=0.1, contrast=0.1), # Added jitter
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load Data
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
               
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print(f"Classes found: {class_names}")
print(f"Dataset sizes: {dataset_sizes}")

Classes found: ['Clinical', 'Histopathological']
Dataset sizes: {'train': 806, 'val': 202}


In [3]:
# Model Setup: ResNet18
# We use a pretrained model for faster convergence and better feature extraction
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer for 2 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

c:\Users\user\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [5]:
# Run Training
model_ft = train_model(model, criterion, optimizer, scheduler, num_epochs=NUM_EPOCHS)

Epoch 0/19
----------
train Loss: 0.2224 Acc: 0.9293
val Loss: 0.0138 Acc: 1.0000

Epoch 1/19
----------
train Loss: 0.0150 Acc: 1.0000
val Loss: 0.0052 Acc: 1.0000

Epoch 2/19
----------
train Loss: 0.0065 Acc: 1.0000
val Loss: 0.0042 Acc: 1.0000

Epoch 3/19
----------
train Loss: 0.0081 Acc: 1.0000
val Loss: 0.0030 Acc: 1.0000

Epoch 4/19
----------
train Loss: 0.0081 Acc: 0.9988
val Loss: 0.0023 Acc: 1.0000

Epoch 5/19
----------
train Loss: 0.0057 Acc: 1.0000
val Loss: 0.0020 Acc: 1.0000

Epoch 6/19
----------
train Loss: 0.0053 Acc: 1.0000
val Loss: 0.0019 Acc: 1.0000

Epoch 7/19
----------
train Loss: 0.0026 Acc: 1.0000
val Loss: 0.0018 Acc: 1.0000

Epoch 8/19
----------
train Loss: 0.0099 Acc: 0.9975
val Loss: 0.0016 Acc: 1.0000

Epoch 9/19
----------
train Loss: 0.0028 Acc: 1.0000
val Loss: 0.0017 Acc: 1.0000

Epoch 10/19
----------
train Loss: 0.0053 Acc: 1.0000
val Loss: 0.0014 Acc: 1.0000

Epoch 11/19
----------
train Loss: 0.0052 Acc: 1.0000
val Loss: 0.0015 Acc: 1.0000

Ep

In [6]:
# Save the Model
torch.save(model_ft.state_dict(), MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")

Model saved to triage_router.pth


alright, now i have already has this model to classfify between histopathological  and clinical model, but here is the issue, how can i upload it and link them to corresponding model such as model a and model b